In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer, HashingVectorizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB,GaussianNB
import itertools
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
import warnings
warnings.filterwarnings("ignore")
from gensim.models import Word2Vec
from sklearn.linear_model import PassiveAggressiveClassifier,LogisticRegression
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

In [17]:
with open(file='train.csv', mode='r') as f:
    df = pd.read_csv(f)
with open(file='test.csv', mode='r') as f:
    df_test = pd.read_csv(f)

In [18]:
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [19]:
df_test

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...
...,...,...,...,...
5195,25995,The Bangladeshi Traffic Jam That Never Ends - ...,Jody Rosen,Of all the dysfunctions that plague the world’...
5196,25996,John Kasich Signs One Abortion Bill in Ohio bu...,Sheryl Gay Stolberg,WASHINGTON — Gov. John Kasich of Ohio on Tu...
5197,25997,"California Today: What, Exactly, Is in Your Su...",Mike McPhate,Good morning. (Want to get California Today by...
5198,25998,300 US Marines To Be Deployed To Russian Borde...,NaN,« Previous - Next » 300 US Marines To Be Deplo...


In [20]:
df = df.dropna()
df.reset_index(inplace = True)

In [21]:
df


,index,id,title,author,text,label
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [22]:
X = df.drop('label',axis =1)
Y = df['label']

In [24]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer, HashingVectorizer

from nltk.stem.porter import PorterStemmer
import re

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB,GaussianNB
import itertools
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
import warnings
warnings.filterwarnings("ignore")
from gensim.models import Word2Vec
from sklearn.linear_model import PassiveAggressiveClassifier,LogisticRegression
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

with open(file='train.csv', mode='r') as f:
    df = pd.read_csv(f)
with open(file='test.csv', mode='r') as f:
    df_test = pd.read_csv(f)

df

df_test

df = df.dropna()
df.reset_index(inplace = True)

df


X = df.drop('label',axis =1)
Y = df['label']

ps = PorterStemmer()
corpus = []
words = []
for i in range(0,len(df)):
    review = re.sub('[^a-zA-Z0-9]',' ',df['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    statements = ' '.join(review)
    corpus.append(statements)
    words.append(review)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [26]:
count = CountVectorizer(max_features=5000,ngram_range=(1,3))
X_count = count.fit_transform(corpus).toarray()
tfidf = TfidfVectorizer(max_features=5000,ngram_range=(1,3))
X_tfidf = tfidf.fit_transform(corpus).toarray()

In [27]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 '15 civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay 2 5 million releas everyon arrest dakota access pipelin',
 'fbi clos

In [28]:
X_count.shape

(18285, 5000)

In [29]:
Y =df["label"]

In [30]:
X_train_count,X_test_count,Y_train_count,Y_test_count = train_test_split(X_count,Y,test_size=0.33,random_state = 27)
X_train_tfidf,X_test_tfidf,Y_train_tfidf,Y_test_tfidf = train_test_split(X_tfidf,Y,test_size=0.33,random_state = 27)

In [33]:
count.get_feature_names_out()[:10]

array(['000', '000 email', '000 job', '00pm', '00pm water',
       '00pm water cooler', '01', '10', '10 000', '10 year'], dtype=object)

In [34]:
count.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.int64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 1.0,
 'max_features': 5000,
 'min_df': 1,
 'ngram_range': (1, 3),
 'preprocessor': None,
 'stop_words': None,
 'strip_accents': None,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': None,
 'vocabulary': None}

In [36]:
df_count = pd.DataFrame(X_train_count,columns = tfidf.get_feature_names_out())
df_count.head()

,000,000 email,000 job,00pm,00pm water,00pm water cooler,01,10,10 000,10 year,...,zealand,zero,zika,zika viru,zionist,zone,zone new,zone new york,zu,zuckerberg
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
df_tfidf = pd.DataFrame(X_train_tfidf,columns = tfidf.get_feature_names_out())
df_tfidf.head()

,000,000 email,000 job,00pm,00pm water,00pm water cooler,01,10,10 000,10 year,...,zealand,zero,zika,zika viru,zionist,zone,zone new,zone new york,zu,zuckerberg
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
classfier_count = MultinomialNB(alpha=0.1)
classfier_tfidf = MultinomialNB(alpha=0.1)

In [40]:
classfier_count.fit(X_train_count,Y_train_count)
Y_pred_count = classfier_count.predict(X_test_count)
classfier_tfidf.fit(X_train_tfidf,Y_train_tfidf)
Y_pred_tfidf = classfier_tfidf.predict(X_test_tfidf)

In [41]:
acc_count = accuracy_score(Y_test_count,Y_pred_count)
classification_count = classification_report(Y_test_count,Y_pred_count)
confusion_matrix_count = confusion_matrix(Y_test_count,Y_pred_count)
acc_tfidf = accuracy_score(Y_test_tfidf,Y_pred_tfidf)
classification_tfidf = classification_report(Y_test_tfidf,Y_pred_tfidf)
confusion_matrix_tfidf = confusion_matrix(Y_test_tfidf,Y_pred_tfidf)

In [42]:
print("FOR COUNT VECTORIZER: \n")
print(" \n Accuracy : ",acc_count,"\n","Classification report \n",classification_count,"\n","Confusion matrix \n",confusion_matrix_count,"\n\n\n")
print("FOR TFIDF VECTORIZER: \n")
print(" \n Accuracy : ",acc_tfidf,"\n","Classification report \n",classification_tfidf,"\n","Confusion matrix \n",confusion_matrix_tfidf)

FOR COUNT VECTORIZER: 

 
 Accuracy :  0.9042253521126761 
 Classification report 
               precision    recall  f1-score   support

           0       0.91      0.92      0.92      3390
           1       0.90      0.89      0.89      2645

    accuracy                           0.90      6035
   macro avg       0.90      0.90      0.90      6035
weighted avg       0.90      0.90      0.90      6035
 
 Confusion matrix 
 [[3116  274]
 [ 304 2341]] 



FOR TFIDF VECTORIZER: 

 
 Accuracy :  0.8783761391880696 
 Classification report 
               precision    recall  f1-score   support

           0       0.85      0.95      0.90      3390
           1       0.92      0.79      0.85      2645

    accuracy                           0.88      6035
   macro avg       0.89      0.87      0.87      6035
weighted avg       0.88      0.88      0.88      6035
 
 Confusion matrix 
 [[3217  173]
 [ 561 2084]]


In [43]:
highest_score_count = 0
highest_score_tfidf = 0
print("Choosing Alpha for Count Vectorizer \n")
for alpha in np.arange(0,1,0.1):
    sub_classifier_count = MultinomialNB(alpha=alpha)
    sub_classifier_count.fit(X_train_count,Y_train_count)
    Y_pred_hyper_count = sub_classifier_count.predict(X_test_count)
    score = accuracy_score(Y_test_count,Y_pred_hyper_count)
    if score > highest_score_count:
        highest_score_count = score
        classfier_count = sub_classifier_count
    print("Alpha : {},Score {}".format(alpha,score))
print("\n Choosing Alpha for Tfidf Vectorizer \n")
for alpha in np.arange(0,1,0.1):
    sub_classifier_tfidf = MultinomialNB(alpha=alpha)
    sub_classifier_tfidf.fit(X_train_tfidf,Y_train_tfidf)
    Y_pred_hyper_tfidf = sub_classifier_tfidf.predict(X_test_tfidf)
    score = accuracy_score(Y_test_tfidf,Y_pred_hyper_tfidf)
    if score > highest_score_tfidf:
        highest_score_tfidf = score
        classfier_tfidf = sub_classifier_tfidf
    print("Alpha : {},Score {}".format(alpha,score))
print("\n\n Best Alpha value for count vectorizer:",classfier_count)
print("\n Best Alpha value for tfidf vectorizer:" ,classfier_tfidf)

Choosing Alpha for Count Vectorizer 

Alpha : 0.0,Score 0.5617232808616405
Alpha : 0.1,Score 0.9042253521126761
Alpha : 0.2,Score 0.9028997514498758
Alpha : 0.30000000000000004,Score 0.9032311516155758
Alpha : 0.4,Score 0.903893951946976
Alpha : 0.5,Score 0.9028997514498758
Alpha : 0.6000000000000001,Score 0.9025683512841757
Alpha : 0.7000000000000001,Score 0.9022369511184756
Alpha : 0.8,Score 0.9024026512013256
Alpha : 0.9,Score 0.9020712510356255

 Choosing Alpha for Tfidf Vectorizer 

Alpha : 0.0,Score 0.5617232808616405
Alpha : 0.1,Score 0.8783761391880696
Alpha : 0.2,Score 0.8793703396851699
Alpha : 0.30000000000000004,Score 0.8810273405136703
Alpha : 0.4,Score 0.8818558409279205
Alpha : 0.5,Score 0.88003314001657
Alpha : 0.6000000000000001,Score 0.8792046396023198
Alpha : 0.7000000000000001,Score 0.8792046396023198
Alpha : 0.8,Score 0.8783761391880696
Alpha : 0.9,Score 0.8783761391880696


 Best Alpha value for count vectorizer: MultinomialNB(alpha=0.1)

 Best Alpha value for tfi

In [45]:
feature_names_count = count.get_feature_names_out()
feature_names_tfidf = tfidf.get_feature_names_out()

In [47]:
sub_classifier_count.feature_log_prob_

array([[ -7.30786336, -11.45451921,  -9.28229594, ...,  -9.28229594,
        -11.45451921,  -9.98818215],
       [ -6.79717072,  -8.41465697,  -9.64248099, ..., -10.81255224,
         -8.52114045, -10.06533784]])

In [49]:

print("\n Fake words from Count Vectorizer \n")
for i in (sorted(zip(sub_classifier_count.feature_log_prob_[0],feature_names_count))[:10]): # Use feature_log_prob_ instead of coef_
          print(i,"\n")
print("\n Fake words from Tfidf Vectorizer \n")
for i in (sorted(zip(sub_classifier_tfidf.feature_log_prob_[0],feature_names_tfidf))[:10]): # Use feature_log_prob_ instead of coef_
    print(i,"\n")


 Fake words from Count Vectorizer 

(-11.454519214081834, '000 email') 

(-11.454519214081834, '00pm') 

(-11.454519214081834, '00pm water') 

(-11.454519214081834, '00pm water cooler') 

(-11.454519214081834, '01') 

(-11.454519214081834, '100percentfedup') 

(-11.454519214081834, '100percentfedup com') 

(-11.454519214081834, '11 16') 

(-11.454519214081834, '11 2016') 

(-11.454519214081834, '2006') 


 Fake words from Tfidf Vectorizer 

(-10.258388662446865, '000 email') 

(-10.258388662446865, '00pm') 

(-10.258388662446865, '00pm water') 

(-10.258388662446865, '00pm water cooler') 

(-10.258388662446865, '01') 

(-10.258388662446865, '100percentfedup') 

(-10.258388662446865, '100percentfedup com') 

(-10.258388662446865, '11 16') 

(-10.258388662446865, '11 2016') 

(-10.258388662446865, '2006') 



In [51]:

print("\n Real words from Count Vectorizer \n")
for i in (sorted(zip(sub_classifier_count.feature_log_prob_[1],feature_names_count),reverse=True)[:10]): # Use feature_log_prob_ instead of coef_ and access the probabilities for class 1 (Real news)
          print(i,"\n")
print("\n Real words from Tfidf Vectorizer \n")
for i in (sorted(zip(sub_classifier_tfidf.feature_log_prob_[1],feature_names_tfidf),reverse=True)[:10]): # Use feature_log_prob_ instead of coef_ and access the probabilities for class 1 (Real news)
    print(i,"\n")


 Real words from Count Vectorizer 

(-3.991929556794031, 'trump') 

(-4.279248014915401, 'hillari') 

(-4.408426317231802, 'clinton') 

(-4.911437970465705, 'elect') 

(-5.18213727028834, 'new') 

(-5.206341759702595, 'comment') 

(-5.304964893722169, 'video') 

(-5.332376386139956, 'war') 

(-5.374956138909219, 'hillari clinton') 

(-5.404384451839828, 'us') 


 Real words from Tfidf Vectorizer 

(-4.934778691096527, 'hillari') 

(-4.960884523428452, 'trump') 

(-5.103351190132023, 'clinton') 

(-5.423377763536257, 'elect') 

(-5.732489702949988, 'comment') 

(-5.74427338205135, 'war') 

(-5.750641633945172, 'video') 

(-5.805652192713543, 'us') 

(-5.866000020369331, 'hillari clinton') 

(-5.869371455352839, 'fbi') 



In [52]:
passive_classifier_count = PassiveAggressiveClassifier(n_iter_no_change = 50)
passive_classifier_count.fit(X_train_count,Y_train_count)
Y_pred_count_passive = passive_classifier_count.predict(X_test_count)
passive_classifier_tfidf = PassiveAggressiveClassifier(n_iter_no_change = 50)
passive_classifier_tfidf.fit(X_train_tfidf,Y_train_tfidf)
Y_pred_tfidf_passive = passive_classifier_count.predict(X_test_tfidf)

In [53]:
acc_count_passive = accuracy_score(Y_test_count,Y_pred_count_passive)
classification_count_passive = classification_report(Y_test_count,Y_pred_count_passive)
confusion_matrix_count_passive = confusion_matrix(Y_test_count,Y_pred_count_passive)
acc_tfidf_passive = accuracy_score(Y_test_tfidf,Y_pred_tfidf_passive)
classification_tfidf_passive = classification_report(Y_test_tfidf,Y_pred_tfidf_passive)
confusion_matrix_tfidf_passive = confusion_matrix(Y_test_tfidf,Y_pred_tfidf_passive)

In [54]:
print("FOR COUNT VECTORIZER - Passive Aggressive Model: \n")
print(" \n Accuracy : ",acc_count_passive,"\n","Classification report \n\n",classification_count_passive,"\n","Confusion matrix \n\n",confusion_matrix_count_passive,"\n\n\n")
print("FOR TFIDF VECTORIZER - Passive Aggressive Model: \n")
print(" \n Accuracy : ",acc_tfidf_passive,"\n","Classification report \n\n",classification_tfidf_passive,"\n","Confusion matrix \n\n",confusion_matrix_tfidf_passive)

FOR COUNT VECTORIZER - Passive Aggressive Model: 

 
 Accuracy :  0.9115161557580779 
 Classification report 

               precision    recall  f1-score   support

           0       0.93      0.92      0.92      3390
           1       0.89      0.91      0.90      2645

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035
 
 Confusion matrix 

 [[3107  283]
 [ 251 2394]] 



FOR TFIDF VECTORIZER - Passive Aggressive Model: 

 
 Accuracy :  0.5373653686826844 
 Classification report 

               precision    recall  f1-score   support

           0       0.99      0.18      0.30      3390
           1       0.49      1.00      0.65      2645

    accuracy                           0.54      6035
   macro avg       0.74      0.59      0.48      6035
weighted avg       0.77      0.54      0.46      6035
 
 Confusion matrix 

 [[ 607 2783]
 [   9 2636]]


In [55]:
stopwords.words("english")

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [56]:
wordvec = Word2Vec(words,min_count=1)

In [58]:
vocab = wordvec.wv.key_to_index

In [59]:
vocab

{'new': 0,
 'time': 1,
 'york': 2,
 'trump': 3,
 'breitbart': 4,
 'hillari': 5,
 'clinton': 6,
 'donald': 7,
 'elect': 8,
 'u': 9,
 'say': 10,
 'obama': 11,
 'presid': 12,
 'state': 13,
 'video': 14,
 'war': 15,
 'vote': 16,
 'russia': 17,
 'report': 18,
 'us': 19,
 'comment': 20,
 'attack': 21,
 'world': 22,
 'year': 23,
 'american': 24,
 'day': 25,
 'news': 26,
 'call': 27,
 'email': 28,
 'fbi': 29,
 'hous': 30,
 'america': 31,
 'get': 32,
 'kill': 33,
 'white': 34,
 'polic': 35,
 'media': 36,
 'campaign': 37,
 'plan': 38,
 'democrat': 39,
 'protest': 40,
 'support': 41,
 'bill': 42,
 'first': 43,
 'one': 44,
 'brief': 45,
 'make': 46,
 'show': 47,
 'win': 48,
 'even': 49,
 'black': 50,
 'syria': 51,
 'anti': 52,
 '2016': 53,
 'back': 54,
 'take': 55,
 'right': 56,
 'watch': 57,
 'nation': 58,
 'russian': 59,
 'man': 60,
 'like': 61,
 '2': 62,
 'peopl': 63,
 'die': 64,
 'go': 65,
 'investig': 66,
 'break': 67,
 'use': 68,
 'may': 69,
 'muslim': 70,
 '3': 71,
 'million': 72,
 'polit':

In [60]:
similar = wordvec.wv.most_similar('hillari')
similar

[('email', 0.9896851778030396),
 ('fbi', 0.9737340211868286),
 ('reopen', 0.972176730632782),
 ('investig', 0.9713091850280762),
 ('clinton', 0.9699752330780029),
 ('campaign', 0.9557739496231079),
 ('wikileak', 0.9490724205970764),
 ('craze', 0.9466468691825867),
 ('probe', 0.9439448714256287),
 ('foundat', 0.9412039518356323)]

In [61]:
vector = wordvec.wv["hillari"]
vector


array([-0.3266953 ,  0.5480329 , -0.10399053, -0.46380267,  0.01992144,
       -0.9655971 ,  0.732617  ,  0.5470933 , -0.27884534, -0.23896074,
       -0.06137403, -1.073779  ,  0.4174788 , -0.09314923, -0.51347107,
       -0.18141112,  0.31900436, -1.4402106 , -0.26874426, -1.3393667 ,
       -0.44250852,  0.7318677 ,  1.3120625 ,  0.11870985, -0.940503  ,
        0.5506124 , -0.76687294, -0.99122775, -0.60585326,  0.12532088,
        0.9276719 , -0.17672113, -0.20106618, -0.74487394, -0.22689007,
        1.1532764 , -0.4795097 , -0.7108194 , -0.7347597 , -1.3295728 ,
        0.68931025, -0.86615705,  0.561368  , -0.301966  ,  0.95116836,
       -0.47483838, -0.69068706, -0.5129341 , -0.31824884,  0.3964848 ,
        0.2604702 , -0.6192165 , -0.04235463,  0.5551635 , -1.0763764 ,
        0.03733016, -0.17015179,  0.18604243, -0.5266593 , -0.2668595 ,
        1.0523473 , -0.20578098,  1.1365198 ,  0.19761652, -0.4678366 ,
        0.2128818 , -0.35267955,  0.13221213, -0.95651925, -0.59

In [62]:
for i,j in zip(words,range(0,len(words))):
    print(j,i)

Streaming output truncated to the last 5000 lines.
13285 ['mitch', 'mcconnel', 'paul', 'ryan', 'meet', 'donald', 'trump', 'promis', 'bold', 'agenda', 'breitbart']
13286 ['susan', 'rice', 'ex', 'obama', 'advis', 'back', 'polit', 'cross', 'hair', 'surveil', 'new', 'york', 'time']
13287 ['fix', 'nbc', 'affili', 'accident', 'post', 'elect', 'result', 'week', 'earli', 'hillari', 'win', 'presid', '42', 'trump', '40']
13288 ['origin', 'unreleas', 'transcript', 'sonia', 'gandhi', 'interview', 'rajdeep', 'sardesai', 'obtain']
13289 ['indian', 'man', 'accus', 'multimillion', 'dollar', 'call', 'center', 'swindl', 'held', 'new', 'york', 'time']
13290 ['7', 'teenag', 'death', 'answer', 'aborigin', 'canadian', 'new', 'york', 'time']
13291 ['fashion', 'cast', 'iron', 'pan', 'today', 'cook', 'new', 'york', 'time']
13292 ['summer', 'sweat', 'hollywood', 'brace', 'possibl', 'worst', 'box', 'offic', 'decad']
13293 ['trump', 'execut', 'order', 'target', 'trade', 'abus', 'amid', 'nafta', 'intrigu', 'breitb

In [63]:
words[562]

[]

In [64]:
rows_to_be_removed = []
for i,j in zip(words,range(0,500000)):
    if i == []:
        rows_to_be_removed.append(j)

In [65]:
len(rows_to_be_removed)

51

In [66]:
series= pd.Series(words)

In [67]:
df = pd.DataFrame(series,columns = ["news"])

In [68]:
df

,news
0,"[hous, dem, aid, even, see, comey, letter, jas..."
1,"[flynn, hillari, clinton, big, woman, campu, b..."
2,"[truth, might, get, fire]"
3,"[15, civilian, kill, singl, us, airstrik, iden..."
4,"[iranian, woman, jail, fiction, unpublish, sto..."
...,...
18280,"[rapper, trump, poster, child, white, supremaci]"
18281,"[n, f, l, playoff, schedul, matchup, odd, new,..."
18282,"[maci, said, receiv, takeov, approach, hudson,..."
18283,"[nato, russia, hold, parallel, exercis, balkan]"


In [69]:
df.drop(df.index[rows_to_be_removed], inplace=True)

In [72]:
def document_vector(doc):
    doc = [word for word in doc if word in wordvec.wv.key_to_index]
    return np.mean(wordvec.wv[doc],axis = 0)

In [73]:
X_wordvec = pd.DataFrame(df.news.apply(document_vector))
X_wordvec.reset_index(inplace=True,drop =True)

In [74]:
X_wordvec

,news
0,"[-0.31673712, 0.5204595, 0.068888664, 0.106516..."
1,"[-0.29703507, 0.53523594, 0.055487495, -0.0207..."
2,"[-0.36956635, 0.59070045, 0.09080523, 0.115930..."
3,"[-0.28280526, 0.46450382, 0.0769967, 0.1061138..."
4,"[-0.22494793, 0.35087723, 0.055298734, 0.07119..."
...,...
18229,"[-0.3054334, 0.43600652, 0.084374815, 0.020141..."
18230,"[-0.39986053, 0.5995348, 0.08082702, 0.2107810..."
18231,"[-0.32055455, 0.4880785, 0.056708455, 0.176615..."
18232,"[-0.19662406, 0.32392928, 0.059707005, 0.06709..."


In [75]:
Y_wordvec = Y.drop(index=rows_to_be_removed)
Y_wordvec = Y_wordvec.reset_index(drop =True)

In [76]:
Y_wordvec

,label
0,1
1,0
2,1
3,1
4,1
...,...
18229,0
18230,0
18231,0
18232,1


In [77]:
X_train_wordvec,X_test_wordvec,Y_train_wordvec,Y_test_wordvec = train_test_split(X_wordvec,Y_wordvec,test_size=0.33,random_state = 27)

In [78]:
lr = LogisticRegression()

In [79]:
X_train_wordvec = X_train_wordvec.squeeze('columns')
X_train_wordvec

,news
4138,"[-0.19085234, 0.2975402, 0.053080935, 0.065841..."
1059,"[-0.26649892, 0.40248433, 0.076202266, 0.05706..."
11557,"[-0.1830473, 0.3513186, 0.0489523, 0.020372467..."
1664,"[-0.3313175, 0.5332896, 0.08504858, 0.14590918..."
14433,"[-0.4078016, 0.6170513, 0.09337776, 0.16900545..."
...,...
15897,"[-0.37347, 0.5682089, 0.068611525, 0.20151958,..."
4848,"[-0.25625217, 0.39160824, 0.06039794, 0.086490..."
14879,"[-0.20364416, 0.31309798, 0.05149125, 0.062240..."
3912,"[-0.437982, 0.67921686, 0.08689481, 0.23944469..."


In [80]:
X_test_wordvec = X_test_wordvec.squeeze('columns')
X_test_wordvec

,news
15801,"[-0.4316772, 0.6668104, 0.0841534, 0.22342268,..."
3345,"[-0.4429934, 0.67573464, 0.08463912, 0.2289948..."
7181,"[-0.21009123, 0.3344122, 0.05312942, 0.0591325..."
5066,"[-0.23389183, 0.37307492, 0.057100154, 0.06898..."
16842,"[-0.20307091, 0.3234297, 0.04353763, 0.0624647..."
...,...
11148,"[-0.28078032, 0.43508133, 0.067816414, 0.10549..."
8915,"[-0.4150026, 0.6433767, 0.08502403, 0.21826822..."
7258,"[-0.4528113, 0.6955582, 0.1348674, 0.13780299,..."
2594,"[-0.23961197, 0.3669721, 0.052452564, 0.050318..."


In [81]:
lr.fit(list(X_train_wordvec),Y_train_wordvec)


LogisticRegression()

In [82]:
Y_pred_wordvec_log = lr.predict(list(X_test_wordvec))

In [83]:
accuracy_score(Y_test_wordvec,Y_pred_wordvec_log)

0.8075772681954138

In [84]:
classfier_wordvec = GaussianNB()

In [85]:
classfier_wordvec.fit(list(X_train_wordvec),Y_train_wordvec)
Y_pred_wordvec_multi = classfier_wordvec.predict(list(X_test_wordvec))

In [86]:
accuracy_score(Y_test_wordvec,Y_pred_wordvec_multi)

0.7540711199734131

In [87]:
df_test

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...
...,...,...,...,...
5195,25995,The Bangladeshi Traffic Jam That Never Ends - ...,Jody Rosen,Of all the dysfunctions that plague the world’...
5196,25996,John Kasich Signs One Abortion Bill in Ohio bu...,Sheryl Gay Stolberg,WASHINGTON — Gov. John Kasich of Ohio on Tu...
5197,25997,"California Today: What, Exactly, Is in Your Su...",Mike McPhate,Good morning. (Want to get California Today by...
5198,25998,300 US Marines To Be Deployed To Russian Borde...,NaN,« Previous - Next » 300 US Marines To Be Deplo...


In [88]:
ps = PorterStemmer()
corpus_test = []
for i in range(0,len(df_test)):
    review = re.sub('[^a-zA-Z0-9]',' ',str(df_test['title'][i]))
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    statements = ' '.join(review)
    corpus_test.append(statements)
    words.append(review)

In [89]:
df_test.isna().sum()

,0
id,0
title,122
author,503
text,7


In [90]:
count_test = CountVectorizer(max_features=5000,ngram_range=(1,3))
X_test_count = count.fit_transform(corpus_test).toarray()

In [91]:
Y_pred_count_passive = passive_classifier_count.predict(X_test_count)

In [92]:
submission= pd.DataFrame({"id":df_test.id,"label":Y_pred_count_passive})
submission.head()

,id,label
0,20800,0
1,20801,1
2,20802,0
3,20803,0
4,20804,1


In [93]:
filename = "submission.csv"
submission.to_csv(filename, index=False)

In [104]:
voc_size = 5000
sent_length=20
embedding_vector_features=40

In [95]:
messages=X.copy()

In [96]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [97]:
messages.reset_index(inplace=True)

In [98]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[3533, 3614, 864, 613, 511, 3671, 4128, 3192, 3955, 1526],
 [4606, 1539, 3368, 3402, 3810, 2229, 3955],
 [4427, 3862, 3448, 1134],
 [1962, 2050, 3320, 4325, 2478, 4246, 1047],
 [932, 3810, 2897, 4827, 4662, 3888, 3810, 2986, 4131, 1688],
 [1585,
  2801,
  399,
  632,
  4659,
  2706,
  1360,
  1182,
  238,
  3221,
  82,
  2446,
  4199,
  3537,
  3955],
 [3672, 1749, 2829, 1553, 4447, 3455, 4257, 3372, 1334, 1256, 1578],
 [1240, 1170, 2496, 3785, 4647, 1048, 2706, 4333, 1334, 1256, 1578],
 [4382, 3826, 1987, 3602, 1169, 437, 2366, 828, 2706, 2199],
 [3688, 2716, 4466, 3772, 1891, 4649, 857, 3612],
 [3995, 4389, 2785, 3912, 742, 4238, 2328, 421, 743, 2750, 3263],
 [2478, 2369, 511, 437, 2706, 4647],
 [1535, 2776, 4148, 4256, 4503, 4706, 1064, 3881, 4500],
 [735, 4205, 2023, 3589, 986, 2422, 3960, 1334, 1256, 1578],
 [2034, 3959, 1369, 2734, 2779, 1334, 1256, 1578],
 [395, 2905, 3753, 2394, 2448, 2831, 4781, 4096, 2149, 3023, 1813, 4488],
 [4735, 3287, 1539],
 [2308, 186, 1871, 3621, 2706

In [99]:
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[3533 3614  864 ...    0    0    0]
 [4606 1539 3368 ...    0    0    0]
 [4427 3862 3448 ...    0    0    0]
 ...
 [1718  739  411 ...    0    0    0]
 [3653 4647   81 ...    0    0    0]
 [2435 1858 2422 ...    0    0    0]]


In [100]:
embedded_docs[0]

array([3533, 3614,  864,  613,  511, 3671, 4128, 3192, 3955, 1526,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [105]:
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [106]:
X_final=np.array(embedded_docs)
y_final=np.array(Y)


In [107]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [108]:
X_train_embed, X_test_embed, Y_train_embed, Y_test_embed = train_test_split(X_final, y_final, test_size=0.33, random_state=27)

In [109]:
model.fit(X_train_embed,Y_train_embed,validation_data=(X_test_embed,Y_test_embed),epochs=20,batch_size=64)


Epoch 1/20
192/192 ━━━━━━━━━━━━━━━━━━━━ 15s 45ms/step - accuracy: 0.7529 - loss: 0.4565 - val_accuracy: 0.9243 - val_loss: 0.1907
Epoch 2/20
192/192 ━━━━━━━━━━━━━━━━━━━━ 15s 71ms/step - accuracy: 0.9452 - loss: 0.1476 - val_accuracy: 0.9251 - val_loss: 0.1857
Epoch 3/20
192/192 ━━━━━━━━━━━━━━━━━━━━ 14s 40ms/step - accuracy: 0.9604 - loss: 0.1132 - val_accuracy: 0.9228 - val_loss: 0.1999
Epoch 4/20
192/192 ━━━━━━━━━━━━━━━━━━━━ 11s 44ms/step - accuracy: 0.9717 - loss: 0.0900 - val_accuracy: 0.9171 - val_loss: 0.2324
Epoch 5/20
192/192 ━━━━━━━━━━━━━━━━━━━━ 13s 56ms/step - accuracy: 0.9774 - loss: 0.0707 - val_accuracy: 0.9135 - val_loss: 0.2609
Epoch 6/20
192/192 ━━━━━━━━━━━━━━━━━━━━ 19s 51ms/step - accuracy: 0.9800 - loss: 0.0613 - val_accuracy: 0.9117 - val_loss: 0.2495
Epoch 7/20
192/192 ━━━━━━━━━━━━━━━━━━━━ 10s 53ms/step - accuracy: 0.9848 - loss: 0.0487 - val_accuracy: 0.9123 - val_loss: 0.2788
Epoch 8/20
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 39ms/step - accuracy: 0.9879 - loss: 0.0402 - v

In [115]:
Y_pred_embed_probs = model.predict(X_test_embed)
Y_pred_embed = (Y_pred_embed_probs > 0.5).astype(int)


189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step


In [116]:
accuracy_score(Y_test_embed,Y_pred_embed)

0.9065451532725767